# Loading a Design Scenario

*Rangekeeper* provides API access to [Speckle](https://speckle.systems/). This
enables loading and extending a 3D design scenario with *Rangekeeper* in order
to produce its financial valuation, as well as execute any automated
decision-making or optimisation processes, and sending the results back to
Speckle.

## Object Model
Before walking through the I/O methods, it is important to understand the
requirements *Rangekeeper* places on how a design scenario must be structured,
so that the `Flow`s and `Stream`s of a DCF Proforma can be appropriately
attributed to the design scenario's objects (especially its representations of
physical spaces, components, and equipment).

### Entities and Relationships
A holistic representation of a real estate development scenario is that it is a
web of inter-related 'entities'; where each entity is a 'thing' and can have
multiple relationships with other entities. For example, a floor is a type
of entity (say, a "Space") and it has relationships with floors above and
below it, as well as relationships to its sub-entities (e.g. rooms), or even
what services it (e.g. the mechanical systems controlling its air conditioning)

```{figure} resources/devModelGraph.jpg
---
width: 100%
name: Real Estate Scenario as a Graph
---
An entire Real Estate Development Scenario is represented as a huge web (graph)
of `Entity`s and `Relationship`s
```

#### `Entity`s
An `Entity` is a ‘thing’ that has substance, or is composed of sub-`Entity`s
that have substance, material, mass, volume, existence, are ‘real’, etc.

#### `Relationship`s
`Relationship`s describe some sort of (directed) association, connection, or
link between `Entity`s. Each `Relationship` is described by its ‘type’; for
instance, an `Entity` can be related to another by virtue of it “being
contained (spatially)” by the other, or it could be “installed before” another
`Entity`.
There can be any number of `Relationship`s between any number of `Entity`s.

#### `Assembly`s
In order to clearly encapsulate ‘groups’ of intuitively-related `Entity`s,
*Rangekeeper* has a concept of an `Assembly`, which is an object that defines
a collection of `Relationship`s and their associated `Entity`s.
`Assembly`s are `Entity`s themselves, too –- so they can also be related to
other `Entity`s.
This means that *Rangekeeper* can traverse from `Assembly` to `Assembly` through
overlapping `Entity`s, in a similar fashion to common-sense conceptualisation
of how a real estate development scenario is structured.

```{figure} resources/devModelAssembly.jpg
---
width: 100%
name: A Diagram of `Assembly`s of `Entity`s
---
An `Assembly` is a non-exclusive collection of `Relationship`s, enabling the
traversal of `Entity`s in the graph.
```

### Definitions
#### Properties of `Entity`s
All `Entity`s have the following properties:
1. **entityId**: a unique & immutable identifier for the `Entity` over the whole of
    its lifetime.
2. **Name**: human-readable text identifying the `Entity`
3. **Type**: an `entityType` (which is a node of a tree of `entityType`s)
4. **Attributes**: a key-value store of specific properties of the `Entity`
   (eg, its area, geometry, material, etc).
There are two key `Attribute`s (stored in the `Attributes` property), where
*Rangekeeper* seeks and records data for its use:
   -  **Events**: a set of time-based occurrences, or experiences, that happen
      in or to the `Entity` (called `Event`s)
   -  **Measurements**: a key-value store of named quantities measuring the
       `Entity` (e.g. its Net Lettable Area (NLA))

In addition, `Assembly`s record a set of `Relationship`s about `Entity`s that
are related to, or in, the `Assembly`.
1. **Relationships**: a set of `Relationship`s

#### Properties of `Relationship`s
Meanwhile, all `Relationship`s have the following properties:
1. **Type**: a `RelationshipType`, which is an item from a list of defined
   `RelationshipType`s
2. **SourceId**: the `Entity` `Id` that is the source of the `Relationship`
3. **TargetId**: the `Entity` `Id` that is the target of the `Relationship`

#### `Event`s
Every `Entity` can exist in both space and time. Most `Entity`s will have (or
its sub-`Entity`s will have, in the case of an `Assembly`) geometry/s that
define the extent of space that the entity exists in.

To represent how an `Entity` exists in time, the `Entity` has `Event`s
recorded against it. For example, an `Event` may record the installation of the
`entity` during construction, or it may record production of revenue during
operation of the real estate project as an asset.

`Flow`s and `Stream`s are subclasses of `Event`s.

```{figure} resources/objModel.jpg
---
width: 100%
name: UML-style diagram of foundation of Rangekeeper's object model
---
Diagram of general overview of the founational concepts of *Rangekeeper*'s
object model, in UML-style.
```

## Rationale and Implications
*Rangekeeper* will use the `Relationship`s defined in the `Assembly`/s of a
design scenario to structure the "drill-downs" and "roll-ups" (ie, the
compositions) of `Flow`s into `Stream`s that are appropriate for the kinds of
summations and metrics that the DCF Proforma model requires.

For example, an office building may produce revenue from multiple tenants. A
tenant may occupy multiple floors, or a part (space) of a single floor.
For *Rangekeeper*, each space would be an `entity`; with its `Events` property
containing a set of `Flow`s and/or `Stream`s that represent revenues or costs
associated with that space. Those may be aggregated into cash flows generated by
the tenant of those spaces, and analysed as such, or they may be aggregated per
floor, or per building, or per project.

This makes it simple to calculate things like the share of total revenue
generated by each tenant; or the share of revenue generated by each floor.
Likewise with costs.

The multi-faceted nature of the [`Entity`-`Relationship` model](
https://en.wikipedia.org/wiki/Entity%E2%80%93relationship_model) enables
multiple pathways for those aggregations to be specified, and thus enables an
efficient and broad ability to query the model for the kinds of analyses
pertinent to real estate valuation.

### Technicalities
To provide a *Rangekeeper* model with the `Entity`-`Relationship` organization,
those need to either be defined in the design scenario itself, or they need to
be specified after receiving and loading the design scenario.

#### Defining `Entity`s and `Relationship`s in the Design Scenario
*Rangekeeper* uses the [Speckle](https://speckle.systems) service and data model
for the exchange of design scenarios, and scaffolds `entity`s off a [Speckle
`Base`](https://speckle.guide/dev/base.html)

It also provides two means to define `entity`s and `Relationship`s:
1. A plugin to [McNeel's Rhinoceros3D Grasshopper]
   (https://www.rhino3d.com/6/new/grasshopper/), that enables both automated and
   manual assignment of `Relationship`s to geometries before their export to
   Speckle; and
2. Manual assignment of `Relationship`s to `Entity`s once they have been loaded
   from a Speckle stream in *Rangekeeper*.

## Example Design Scenario
An example development has been designed and uploaded to Speckle for us to use
as a demonstration of the object model and its use in *Rangekeeper*.

This design scenario is composed of two mixed-use buildings, that both share a
common basement plinth, used for parking and shared mechanical and other
services (e.g. garbage disposal, storage)

To access the design scenario in *Rangekeeper*, we need to load it from Speckle.
First load neccesary libraries:

In [32]:
import os

import pprint as pp
from IPython.display import IFrame

import rangekeeper as rk

### Authenticate and Load the Design scenario from Speckle

In [33]:
# Authenticate with Speckle
speckle = rk.api.Speckle(
    host="speckle.xyz",
    token=os.getenv('SPECKLE_TOKEN')) # Note if you run this notebook yourself, you'll need to set this environment variable



 SpeckleClient( server: https://speckle.xyz, authenticated: True )


The scenario is stored in a Speckle stream, which is a container for a set of
'commits', or versions of the stream.

In [34]:
stream_id = "f5e306e3fa"
commit_id = speckle.get_latest_commit_id(stream_id)

In [35]:
# Load the design scenario
model = speckle.get_commit(stream_id=stream_id) # Providing no commit_id defaults to the latest commit
model

Base(id: b914ed89cf5040b5d05ff14d22e49c39, speckle_type: Base, totalChildrenCount: 2898)

### Inspect the Design Scenario

In [36]:
IFrame("https://speckle.xyz/embed?stream={0}&commit={1}".format(stream_id, commit_id), width='100%', height=800)

In [37]:
# Let's identify the root members of the scenario:
roots = model.get_dynamic_member_names()
roots

['@context', '@scenario']

In this design scenario, there are two members (Speckle Objects) at the root of
the model:
1. '@context', and
2. '@scenario'

```{note}
Speckle Objects whose member names begin with an `@` indicate 'detached' data:
See [The Base Object: Detaching](https://speckle.guide/dev/base.html#detaching)
for more information.
```

For future reference, if we wanted to see what a  they represented, we can pass them in to the Speckle
Viewer's "objects" endpoint with their `id`s:

In [38]:
IFrame("https://speckle.xyz/streams/{0}/objects/{1}".format(stream_id, model['@scenario']['id']), width='100%', height=800)

We can even use the Speckle Viewer's 'filter' functionality to see only the
certain types contained within the spaces of the design scenario (try toggling
the isolation mode for "floor" or "utilities" types):

In [39]:
property_name = 'type'
IFrame('https://speckle.xyz/streams/{0}/commits/{1}?filter=%7B%22propertyInfoKey%22%3A%22{2}%22%7D'.format(
    stream_id,
    commit_id,
    property_name), width='100%', height=800)

### Rangekeeper `Entity`s in the Design Scenario
Of course, not all Speckle Objects are Rangekeeper `Entity`s for us to use in
our valuations and modeling.
To make parsing the design scenario easier, *Rangekeeper* provides some helper
methods to convert the Speckle Objects into Rangekeeper `Entity`s:

In [40]:
# Return any Speckle Objects in the '@scenario' trunk that have 'entityId's:
parsed = rk.api.Speckle.parse(base=model['@scenario'])

# (Recursively) Convert the Speckle Objects into Rangekeeper Entities:
scenario = rk.api.Speckle.to_rk(
    bases=list(parsed.values()),
    name='scenario',
    type='scenario')

# (We can check that it is an Assembly:)
isinstance(scenario, rk.graph.Assembly)

Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.
Existing Entity is an Assembly while new Entity is not. Keeping Assembly.


True

#### Querying the Design Scenario's Object Graph
*Rangekeeper* uses the [NetworkX](https://networkx.org/) library to represent
the object graph of the design scenario, where each `Assembly` is a
`MultiDiGraph` of `Entity` nodes and `Relationship` edges.
This means we can query it pythonically:

In [41]:
# Get the "BuildingA" Assembly:
buildingA = [entity for (entityId, entity) in scenario.get_entities().items() if entity['name'] == 'buildingA'][0]
print('buildingA: {0}'.format(buildingA))

buildingA: Assembly: buildingA
Type: building
Members: ['name', 'type']
Entities: [('eb42cd14-73ab-41fc-99c0-c46631ee1208', {'entity': Assembly: buildingA (Type: building)}), ('655b2dfe-5c18-4d68-95f0-f5d27804430d', {'entity': Assembly: buildingAoffice (Type: space)}), ('47dfa8b6-e213-4099-9f5b-834df0ca34b0', {'entity': Assembly: buildingAretail (Type: space)}), ('da796c3e-94b0-4175-8c6f-ac63e5e60714', {'entity': Assembly: buildingAresidential (Type: space)}), ('08e12c72-4450-424f-8ff4-a4da7246ee05', {'entity': Assembly: buildingAretail (Type: space)}), ('634c5fb4-f204-4287-8c80-f23371056f1c', {'entity': Assembly: buildingAparking (Type: space)}), ('abdf16fd-494e-4d7f-a604-134ce1c004ac', {'entity': Entity: buildingAcores (Type: utilities)})]
Relationships: [('eb42cd14-73ab-41fc-99c0-c46631ee1208', '655b2dfe-5c18-4d68-95f0-f5d27804430d', 'spatiallyContains'), ('eb42cd14-73ab-41fc-99c0-c46631ee1208', '47dfa8b6-e213-4099-9f5b-834df0ca34b0', 'spatiallyContains'), ('eb42cd14-73ab-41fc-99c0-

Because we can also define directed, labelled `Relationship`s between `Entity`s,
we can identify relatives of `Entity`s accordingly:

In [42]:
# Get all relatives of BuildingA where BuildingA is the source of
# a 'spatiallyContains' relationship:
buildingA_containment = buildingA.get_relatives(
    outgoing=True,
    relationship_type='spatiallyContains',
    assembly=scenario)
print('buildingA Containment: \n {0}\n'.format(buildingA_containment))

buildingA Containment: 
 [Assembly: buildingAoffice (Type: space), Assembly: buildingAretail (Type: space), Assembly: buildingAresidential (Type: space), Assembly: buildingAretail (Type: space), Assembly: buildingAparking (Type: space), Entity: buildingAcores (Type: utilities)]



In [43]:
buildingAresi = [entity for entity in buildingA_containment if entity['name'] == 'buildingAresidential'][0]
print('buildingAresidential: {0}'.format(buildingAresi))

buildingAresidential: Assembly: buildingAresidential
Type: space
Members: ['@displayValue', 'renderMaterial', 'name', 'type', 'use']
Entities: [('da796c3e-94b0-4175-8c6f-ac63e5e60714', {'entity': Assembly: buildingAresidential (Type: space)}), ('a987a169-2605-4308-9462-8d22eb6df2be', {'entity': Entity: buildingAresidentialFloor0 (Type: floor)}), ('d2db8f8f-36c9-4a92-a1dd-016376fee217', {'entity': Entity: buildingAresidentialFloor0 (Type: floor)}), ('b0384123-9bec-4c0e-87f6-32abb6a49fb3', {'entity': Entity: buildingAresidentialFloor1 (Type: floor)}), ('bbd83975-af58-492e-8818-ce1968564a05', {'entity': Entity: buildingAresidentialFloor2 (Type: floor)}), ('c78fd645-372b-4a23-a823-98e2f6885b97', {'entity': Entity: buildingAresidentialFloor3 (Type: floor)})]
Relationships: [('da796c3e-94b0-4175-8c6f-ac63e5e60714', 'a987a169-2605-4308-9462-8d22eb6df2be', 'spatiallyContains'), ('da796c3e-94b0-4175-8c6f-ac63e5e60714', 'd2db8f8f-36c9-4a92-a1dd-016376fee217', 'spatiallyContains'), ('da796c3e-94b

We can also visualize the scenario graph as well:

In [45]:
scenario.plot(name='assets/design_scenario')
IFrame(src="./design_scenario.html", width='100%', height=800)

assets/design_scenario.html
